In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd #엑셀화 판다스
import time
from selenium.webdriver.common.keys import Keys #키보드
from bs4 import BeautifulSoup as bs

In [7]:
# 가져온 데이터를 텍스트화 해서 저장하는 리스트들
date_site_list = []
star_list = []
content_list = []
title_list=[]
review_all = []

# 사이트 가기https://www.amazon.com/ref=nav_logo
driver=webdriver.Chrome(ChromeDriverManager().install())
#data 창이 뜨고 나서 그걸 움직이는 것
#가져오고자 하는 url입력
#.get.url("URL")
driver.get("https://www.amazon.com/ref=nav_logo")
driver.maximize_window()
#광고 끄기
try: search=driver.find_element_by_xpath('//*[@id="nav-main"]/div[1]/div/div/div[3]/span[1]/span/input').click()
except:
    print("지역 광고가 없습니다! 다음을 진행해 주세요.")

#검색하기
search=driver.find_element_by_xpath('//*[@id="twotabsearchtextbox"]')
search.clear()
# a = str(input("검색어를 입력하세요: "))
search.send_keys(coffee machine)
search.send_keys(Keys.ENTER)

i=1 #상품검색 페이지 수
while True: 
    print(f"=========== 아직 {i}페이지 입니다 ===========")
    k=0
    for j in range(1,65):
        try:
            #각 페이지에서 제품 사이트 들어가기
            driver.find_element_by_xpath(f'//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[{j}]/div/div/div/div/div[2]/div[1]/h2/a/span').click()
            time.sleep(1)

            soup=bs(driver.page_source, 'html.parser')

            #상품명 가져오기
            title_bs=soup.select('h1>span.a-size-large.product-title-word-break')
            for P in title_bs:
                title_list.append(P.text.strip())

            #모든 리뷰보기로 이동하기 위한 url 가져오기
            AllReview = soup.select("a.a-link-emphasis.a-text-bold") 
            AllReview_url = 'https://www.amazon.com/' + AllReview[0]["href"]

            #모든 리뷰로 이동
            driver.get(AllReview_url) 
            driver.implicitly_wait(5)
            print("넘어갔음")

            x = 1
            while True:
                #모든 리뷰의 페이지 소스 가져오기
                soup = bs(driver.page_source, "lxml") 

                #원하는 태그 삭제하기(번역)
                remove = soup.select("span.a-size-base.review-text.review-text-content > span.cr-translated-review-content.aok-hidden")
                for rm in remove:
                    rm.extract()

                #[Beautiful Soup] CSS를 이용한 코드 가져오기(날짜&장소, 별점, 내용)
                date_site = soup.select('div.a-section.celwidget > span.a-size-base.a-color-secondary.review-date') 
                star = soup.select('div:nth-child(2) > a:nth-child(1) > i > span.a-icon-alt')
                content = soup.select('span.a-size-base.review-text.review-text-content > span') 

                #[Beautiful Soup] 가져온 코드 Text화해서 리스트에 추가하기
                for b in range(len(date_site)):
                    date_site_list.append(date_site[b].text.replace("Reviewed in ", ""))
                    star_list.append(star[b].text.replace(" out of 5 stars", ""))
                    content_list.append(content[b].text)
              
                review_all.append(list(date_site_list))
                review_all.append(list(star_list))
                review_all.append(list(content_list))

                # 리뷰 다음페이지로 이동하기 / 더이상 페이지 없으면 멈추기
                try:    
                    x += 1            
                    NextPage = driver.find_element_by_xpath('//*[@id="cm_cr-pagination_bar"]/ul/li[2]/a') 
                    NextPage.click()
                    time.sleep(2)
                    driver.refresh()
                    driver.implicitly_wait(5)
                    time.sleep(2)
                except:
                    print(f"{x-1}페이지에서 끝냅니다.")
                    break

        # 리뷰를 빠져나올 때 첫번째 검색한 페이지로 가는 부분에서 오류가 발생함. 전 페이지로 가면 리뷰페이지만 있을 수 밖에...    
        # driver.back()
        # print("98번째 지나감")
            k=j+1

        except:
            print(f"{j}번째 상품이 없음.") # 해당하지 않은 제품의 개수를 확인하기 위한 출력문
            pass
    
    i += 1
    # 원하는 상품을 검색한 페이지에서 다음페이지로 넘어가는 코드 #2,3,4, .... 페이지로
    if i == 1: # 1페이지에서 2페이지에서 xpath값 지정
        num1 = 29
        num2 = 1
        driver.find_element_by_xpath(f'//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[{num1}]/div/div/span/a[{num2}]').click()
        time.sleep(1)
    elif i == 2: # 2페이지에서 3페이지에서 xpath값 지정
        num1 = 28
        num2 = 3
        driver.find_element_by_xpath(f'//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[{num1}]/div/div/span/a[{num2}]').click()
        time.sleep(1) 
    elif i == 3:  # 3페이지에서 4페이지에서 xpath값 지정
        num1 = 28
        num2 = 4
        driver.find_element_by_xpath(f'//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[{num1}]/div/div/span/a[{num2}]').click()
        time.sleep(1)
    elif i == 4: # 4페이지에서 5페이지에서 xpath값 지정
        num1 = 27
        num2 = 5
        driver.find_element_by_xpath(f'//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[{num1}]/div/div/span/a[{num2}]').click()
        time.sleep(1) 
    else:  
        num1 = 28
        num2 = 4
        driver.find_element_by_xpath(f'//*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[{num1}]/div/div/span/a[{num2}]').click()
        # 2페이지 //*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[67]/div/div/span/a[1]
        # div.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 > div > span:nth-child(4) > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(67) > div > div > span > a:nth-child(3)
        # 3페이지 //*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[66]/div/div/span/a[3]
        # div.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 > div > span:nth-child(4) > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(66) > div > div > span > a:nth-child(4)
        # 4페이지 //*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[65]/div/div/span/a[4]
        # div.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 > div > span:nth-child(4) > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(65) > div > div > span > a:nth-child(5)
        # 5페이지 //*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[66]/div/div/span/a[5]
        # div.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 > div > span:nth-child(4) > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(66) > div > div > span > a:nth-child(6)
        # 6페이지 //*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[66]/div/div/span/a[4]
        # div.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 > div > span:nth-child(4) > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(66) > div > div > span > a:nth-child(6)
        # 7페이지 //*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[66]/div/div/span/a[4]
        # div.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 > div > span:nth-child(4) > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(65) > div > div > span > a:nth-child(6)
        # 다음페이지 없음 //*[@id="search"]/div[1]/div[1]/div/span[3]/div[2]/div[35]/div/div/span/span[3]
        # div.s-matching-dir.sg-col-16-of-20.sg-col.sg-col-8-of-12.sg-col-12-of-16 > div > span:nth-child(4) > div.s-main-slot.s-result-list.s-search-results.sg-row > div:nth-child(36) > div > div > span > span.s-pagination-item.s-pagination-next.s-pagination-disabled
        time.sleep(1)


    # if k==0:#광고문 제거선언
    #     print("91번에서 오류")
    #     break

102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
넘어갔음
13페이지에서 끝냅니다.
98번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감
102번째 지나감


TypeError: unsupported operand type(s) for +: 'Tag' and 'int'

In [8]:
print(len(date_site_list), len(star_list), len(content_list))

#print( "date_site", len(date_site),  "star", len(star), "content", len(content))

121 121 121


In [6]:
k

0

In [232]:
print(len(date_site_list), len(star_list), len(content_list))

print( "date_site", len(date_site),  "star", len(star), "content", len(content))

x = 1
for i in content:
    try:    
        print(f"================{x}번째=======================")
        print(i.text)
        x += 1
    except:
        print(f"{i}번째 오류 \n")



31 31 31
date_site 1 star 1 content 1
================1번째=======================
This machine makes perfect expresso--rich dark coffee with a tan foam.  Easy to clean the coffee part.  The milk frother is another story.  Very difficult to use correctly and also a pain to clean.  This appliance would be a 5 star but for the frother.  I use a separate tool to froth the milk/half & half.  I recommend this machine for  expresso performance- at a good price point.


In [58]:
# 가져온 데이터를 텍스트화 해서 저장하는 리스트들
date_site_list = []
star_list = []
content_list = []
title_list=[]
review_all = []

# 사이트 가기https://www.amazon.com/ref=nav_logo
driver=webdriver.Chrome(ChromeDriverManager().install())
#data 창이 뜨고 나서 그걸 움직이는 것
#가져오고자 하는 url입력
#.get.url("URL")
driver.get("https://www.amazon.com/s?k=coffee+machine&crid=3G3G5RBAIJZCN&sprefix=%2Caps%2C211&ref=nb_sb_ss_recent_1_0_recent")
driver.maximize_window()

#광고 끄기
try: search=driver.find_element_by_xpath('//*[@id="nav-main"]/div[1]/div/div/div[3]/span[1]/span/input').click()
except:
    print("지역 광고가 없습니다! 다음을 진행해 주세요.")

soup = bs(driver.page_source, "lxml") 

# driver.close()

content = soup.select('''h2.a-size-mini.a-spacing-none.a-color-base.s-line-clamp-4 
> a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal> span.a-size-base.-color-base.a-text-normal''') 

# [Beautiful Soup] 가져온 코드 Text화해서 리스트에 추가하기
for b in content:
    print(b.text)
# NextPage = driver.find_element_by("a-size-base a-color-base a-text-normal") 
# NextPage.click()

# driver.close()

In [57]:
len(content)

0